In [251]:
import numpy as np
import pandas as pd

In [252]:
df = pd.read_csv("./data2.csv", names = ["StartTime", "IPVersion", "IHL", "TOS", "Len", "ID", 
                                         "Frag", "TTL", "Protocol", "SrcIP", "DstIP", "IPFlags", "IPchecksum",
                                         "SrcPort", "DstPort", "TCPSeq/UDPlen",
                                          "TCPAck/UDPcksm", "DataOff", "Reserved", "TCPFlags", "Window", "TCPcksum", "UrgPtr"])

In [253]:
df.fillna(0,inplace=True)
df["TCPFlags"] = df["TCPFlags"].astype(int)

In [254]:
newdf = df.groupby(["SrcIP","DstIP","SrcPort","DstPort","Protocol"])

In [258]:
def func(x):
    return {"StartTime": {"Duration": x.max() - x.min(), 
                                    "StartTime": x.min(), 
                                    "EndTime": x.max()},
#                       "TOS": {"TOS": x.max()}, 
#                       "Len": {"MeanLength": x.mean(), 
#                               "TotalLength": x.sum()}, 
#                       "IPFlags": {"IPFlags": x},#np.bitwise_or.reduce(x)},
#                       "TTL": {"TTL": x.mean()}, 
#                       "DataOff": {"DataOff": x.max()}, 
#                       "TCPFlags": {"TCPFlags": x},#np.bitwise_or.reduce(x)}, 
#                       "Window": {"Window": x.mean()}, 
#                       "PktCount": x.count()
                     }

finalfd = newdf.agg({"StartTime": {"Duration": lambda x: x.max() - x.min(), 
                                   "StartTime": lambda x: x.min(), 
                                   "EndTime": lambda x: x.max(),
                                   "PktRate": lambda x: x.count()/ (x.max() - x.min())},
                      "TOS": {"TOS": lambda x: x.max()}, 
                      "Len": {"MeanLength": lambda x: x.mean(), 
                              "TotalLength": lambda x: x.sum(), 
                              "PktCount": lambda x: x.count()}, 
                      "IPFlags": {"IPFlags": lambda x: np.bitwise_or.reduce(x.as_matrix())},
                      "TTL": {"TTL": lambda x: x.mean()}, 
                      "DataOff": {"DataOff": lambda x: x.max()}, 
                      "TCPFlags": {"TCPFlags": lambda x: np.bitwise_or.reduce(x.as_matrix()) if x.any() else 0}, 
                      "Window": {"Window": lambda x: x.mean()}, 
                     })

/usr/lib/python3.6/site-packages/pandas/core/groupby.py:4291: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)
/usr/lib/python3.6/site-packages/ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in true_divide


In [259]:
finalfd

StartTime  \
                                                           Duration   
SrcIP          DstIP           SrcPort DstPort Protocol               
0.0.0.0        255.255.255.255 68      67      17        292.129199   
109.109.254.2  192.168.4.119   80      3008    6           0.395486   
                                       3274    6           0.389235   
                                       3280    6           0.396741   
113.20.6.154   192.168.4.120   80      2830    6           2.686859   
                                       3012    6           2.185649   
                                       3034    6           3.038611   
                                       3045    6           0.748230   
                                       3054    6           1.252434   
                                       3061    6           4.455598   
                                       3062    6           4.201579   
                                       3067    6           1.255147   
                                       3076    6           0.701836   
                                       3082    6           4.256781   
113.32.19.131  192.168.2.108   80      1910    6          19.364072   
                                       1932    6          42.929258   
                                       1969    6          10.903220   
                                       1986    6           7.054688   
                                       2010    6           5.940310   
                                       2023    6           8.612400   
                                       2043    6           9.596162   
                                       2062    6          10.062955   
                                       2080    6           5.977968   
                                       2095    6           3.309014   
                                       2107    6           6.830260   
                                       2111    6           0.512688   
                                       2119    6           0.761501   
                                       2212    6           3.945721   
                                       2230    6           4.673059   
                                       2244    6           0.527660   
...                                                             ...   
98.139.50.166  192.168.1.103   443     3940    6          15.199681   
                                       3958    6          15.381761   
                                       3981    6          15.186675   
                                       4041    6          15.375576   
                                       4076    6          15.587903   
                                       4130    6          15.275337   
                                       4157    6          15.181903   
                                       4245    6          15.303577   
                                       4288    6          15.322081   
                                       4309    6          19.380761   
                                       4427    6          15.540990   
                                       4499    6          15.457741   
               192.168.1.104   443     18575   6          15.289839   
               192.168.3.117   443     1135    6          19.288549   
98.142.102.30  192.168.3.114   80      1630    6          20.158697   
                                       1650    6          24.012597   
                                       1868    6          22.004558   
98.158.176.206 192.168.1.101   80      1124    6           2.292794   
                                       2757    6           2.050646   
                                       4582    6           2.295040   
                                       4740    6           2.308529   
               192.168.1.102   80      3461    6           2.051729   
               192.168.1.103   80      2904    6          12.752183   
           

In [260]:
finalfd.to_csv("flowfile.csv", sep=",")